# Project Inscriptions -- Exploratory Data Analysis

**Johnnatan Messias**, April 2024


In [1]:
import os
from tqdm.notebook import tqdm
import polars as pl
import json
import plotly.graph_objects as go

In [2]:
import sys
code_dir = os.path.realpath(os.path.join(os.getcwd(), "..", "src"))

sys.path.append(code_dir)

In [50]:
from plot_utils import get_plotly_layout
from plot_utils import colors
width, height = 800, 450

ImportError: cannot import name 'get_plotly_layout' from 'plot_utils' (/Users/kris/Library/Python/3.9/lib/python/site-packages/plot_utils/__init__.py)

In [51]:
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

width, height = 800, 450
layout = go.Layout(
    template='simple_white',
    font=dict(size=18, family='Clear Sans'),
    margin=go.layout.Margin(
        l=10,  # left margin
        r=10,  # right margin
        b=10,  # bottom margin
        t=10  # top margin
    ),
    width=width,
    height=height,
    xaxis=dict(minor_ticks="inside", showgrid=True,
               griddash='dash', minor_griddash="dot"),
    yaxis=dict(minor_ticks="inside", showgrid=True,
               griddash='dash', minor_griddash="dot")
)

In [52]:
colors = {'red': '#ee443a', 'blue': '#42bbf1', 'dark_blue': '#1a4fec',
          'green': '#50be61', 'grey': '#b7b7b7', 'orange': '#f28222', 'purple': '#6e18ee', 'brown': '#a65628', 'pink': '#ef4793',
          'yellow': '#f8c94c', 'black': '#000000', 'white': '#ffffff', 'light_blue': '#a6cee3', 'light_green': '#b2df8a',
          'light_grey': '#999999', 'light_orange': '#fdbf6f', 'light_purple': '#cab2d6', 'light_brown': '#ffff99', 'light_pink': '#1f78b4',
          'light_yellow': '#fb9a99', 'light_black': '#e31a1c', 'light_white': '#33a02c', 'gold': '#ff7f00', 'silver': '#b2df8a'}
styles = ['-', '--', ':', '-.']
percentiles = [.01, .05, .1, .2, .25, .50, .75, .8, .9, .95, .99]
linestyles = ['dotted', 'dotted', 'solid', 'dashdot', 'dashed', 'solid']

In [3]:
# Existing dataset dir
data_dir = '../data/'

# Existing plots dir
plots_dir = data_dir+'/plots/'
os.makedirs(data_dir, exist_ok=True)
os.makedirs(plots_dir, exist_ok=True)

In [ ]:
inscriptions_tag = '0x646174613a'

## Exploratory Data Analysis


In [4]:
def parse_input_data(data):
    response = dict()
    if data is None:
        response = None
    else:
        if data.startswith('data:,{'):
            response['type'] = 'json'
            response['data'] = ','.join(data.split(',')[1:])
        elif data.startswith('data:image/png;'):
            response['type'] = 'image/png'
            response['data'] = ','.join(data.split(',')[1:])
        elif data.startswith('data:application/json,'):
            response['type'] = 'application/json'
            response['data'] = ','.join(data.split(',')[1:])
        elif data.startswith('data:application/json,'):
            response['type'] = 'application/json'
            response['data'] = ','.join(data.split(',')[1:])
        else:
            response['type'] = 'unknown'
            response['data'] = data
    return response

In [5]:
# Loading inscriptions dataframe from a file
inscriptions_df = pl.scan_parquet(
    data_dir+'inscriptions_df.parquet').collect(streaming=True)
print("There are {} inscriptions in our dataset.".format(
    inscriptions_df.shape[0]))
inscriptions_df.head()

There are 17060492 inscriptions in our dataset.


block_number,tx_hash,tx_input_data,issuer,receiver,timestamp,gas_used,gas_effective_price,fees,tx_status,decoded_input_data
i64,str,str,str,str,datetime[μs],i64,i64,f64,i64,str
6332862,"""0x2359c19cc715…","""0x646174613a2c…","""0x86f04d8f599a…","""0x86f04d8f599a…",2023-06-18 02:04:06,215530,250000000,0.000054,1,"""data:,dashboar…"
6351363,"""0x9fb936db1466…","""0x646174613a69…","""0x39ebe965aff2…","""0x488bd13f16a2…",2023-06-18 07:13:34,221181,250000000,0.000055,1,"""data:image/png…"
6351394,"""0x364528a9e973…","""0x646174613a69…","""0x39ebe965aff2…","""0x488bd13f16a2…",2023-06-18 07:14:05,148688,250000000,0.000037,1,"""data:image/png…"
6351410,"""0x0d746c0320ff…","""0x646174613a69…","""0x39ebe965aff2…","""0x488bd13f16a2…",2023-06-18 07:14:21,220987,250000000,0.000055,1,"""data:image/png…"
6351431,"""0xcd99fb93cee9…","""0x646174613a69…","""0x39ebe965aff2…","""0x488bd13f16a2…",2023-06-18 07:14:42,147604,250000000,0.000037,1,"""data:image/png…"


In [7]:
print("There are {} unique transactions in our dataset.".format(
    inscriptions_df['tx_hash'].n_unique()))
print("There are {} unique issuers in our dataset.".format(
    inscriptions_df['issuer'].n_unique()))
print("There are {} unique blocks in our dataset.".format(
    inscriptions_df['block_number'].n_unique()))
print("The average number of inscriptions per block is {} inscriptions.".format(round(
    inscriptions_df['tx_hash'].n_unique()/inscriptions_df['block_number'].n_unique(), 2)))

print("The minimum and max block numbers containing inscriptions are: {} and {}.".format(
    inscriptions_df['block_number'].min(), inscriptions_df['block_number'].max()))

print("The minimum timestamp is {} and the maximum timestamp is {}.".format(
    inscriptions_df['timestamp'].min(), inscriptions_df['timestamp'].max()))

There are 17054466 unique transactions in our dataset.
There are 470864 unique issuers in our dataset.
There are 2713534 unique blocks in our dataset.
The average number of inscriptions per block is 6.28 inscriptions.
The minimum and max block numbers containing inscriptions are: 6332862 and 29799866.
The minimum timestamp is 2023-06-18 02:04:06 and the maximum timestamp is 2024-03-25 02:19:14.


In [6]:
tokens_df = (inscriptions_df
             .filter(pl.col('decoded_input_data').str.starts_with('data:,{'))
             .with_columns(pl.col('decoded_input_data').str.slice(6).alias('decoded_input_data')))
print("There are {} tokens.".format(tokens_df.shape[0]))
tokens_df.head()

There are 16863729 tokens.


block_number,tx_hash,tx_input_data,issuer,receiver,timestamp,gas_used,gas_effective_price,fees,tx_status,decoded_input_data
i64,str,str,str,str,datetime[μs],i64,i64,f64,i64,str
6359996,"""0x51c592fc710d…","""0x646174613a2c…","""0x171c8be8b926…","""0x171c8be8b926…",2023-06-18 09:38:14,219331,250000000,0.000055,1,"""{""p"":""erc-20"",…"
6360292,"""0xb453be720aee…","""0x646174613a2c…","""0x61fd0d043d51…","""0x61fd0d043d51…",2023-06-18 09:43:10,221045,250000000,0.000055,1,"""{""p"":""erc-20"",…"
6360319,"""0xd6e8059343a9…","""0x646174613a2c…","""0x34e4915528dc…","""0x34e4915528dc…",2023-06-18 09:43:37,220718,250000000,0.000055,1,"""{""p"":""erc-20"",…"
6360372,"""0x91e1c6e17a3f…","""0x646174613a2c…","""0x61fd0d043d51…","""0x61fd0d043d51…",2023-06-18 09:44:30,219600,250000000,0.000055,1,"""{""p"":""erc-20"",…"
6360489,"""0x0c3e0555c76f…","""0x646174613a2c…","""0x531e2b809625…","""0x531e2b809625…",2023-06-18 09:46:28,220008,250000000,0.000055,1,"""{""p"":""erc-20"",…"


In [9]:
inscriptions_df.shape[0] - tokens_df.shape[0]

196763

In [7]:
del inscriptions_df

In [ ]:
# if not os.path.exists(data_dir+'inscriptions_tokens.parquet'):
#     print("File does not exist: Parsing inscriptions -- tokens")
#     df = list()
#     pbar = tqdm(total=tokens_df.shape[0],
#                 desc='Parsing inscriptions -- tokens')
#     for row in tokens_df.iter_rows(named=True):
#         pbar.update(1)
#         try:
#             input_in_json = json.loads(row['decoded_input_data'])
#         except:
#             continue
#         data = {
#             'block_number': row['block_number'],
#             'tx_hash': row['tx_hash'],
#             'issuer': row['issuer'],
#             'timestamp': row['timestamp'],
#             'gas_used': row['gas_used'],
#             'gas_effective_price': row['gas_effective_price'],
#             'fees': row['fees'],
#             'tx_status': row['tx_status'],
#         }
#         data.update(input_in_json)
#         df.append(data)
#     pbar.close()
#     print("There are {} tokens.".format(len(df)))
#     df = pl.DataFrame(df)
#     df.write_parquet(data_dir+'inscriptions_tokens.parquet')
# else:
#     print("Loading inscriptions_tokens.parquet")
#     df = pl.read_parquet(data_dir+'inscriptions_tokens.parquet')
#     print("There are {} tokens.".format(df.shape[0]))
#     df.head()

In [8]:
inscriptions_parsed = (tokens_df
                       .with_columns([
                           pl.col('decoded_input_data').str.json_path_match(
                               r"$.p").alias('p'),
                           pl.col('decoded_input_data').str.json_path_match(
                               r"$.op").alias('op'),
                           pl.col('decoded_input_data').str.json_path_match(
                               r"$.tick").alias('tick'),
                           pl.col('decoded_input_data').str.json_path_match(
                               r"$.amt").alias('amt'),
                           pl.col('decoded_input_data').str.json_path_match(
                               r"$.tx").alias('tx'),
                           pl.col('decoded_input_data').str.json_path_match(
                               r"$.price").alias('price'),

                       ]))
inscriptions_parsed

block_number,tx_hash,tx_input_data,issuer,receiver,timestamp,gas_used,gas_effective_price,fees,tx_status,decoded_input_data,p,op,tick,amt,tx,price
i64,str,str,str,str,datetime[μs],i64,i64,f64,i64,str,str,str,str,str,str,str
6359996,"""0x51c592fc710d…","""0x646174613a2c…","""0x171c8be8b926…","""0x171c8be8b926…",2023-06-18 09:38:14,219331,250000000,0.000055,1,"""{""p"":""erc-20"",…","""erc-20""","""mint""","""eths""","""1000""",null,null
6360292,"""0xb453be720aee…","""0x646174613a2c…","""0x61fd0d043d51…","""0x61fd0d043d51…",2023-06-18 09:43:10,221045,250000000,0.000055,1,"""{""p"":""erc-20"",…","""erc-20""","""mint""","""eths""","""1000""",null,null
6360319,"""0xd6e8059343a9…","""0x646174613a2c…","""0x34e4915528dc…","""0x34e4915528dc…",2023-06-18 09:43:37,220718,250000000,0.000055,1,"""{""p"":""erc-20"",…","""erc-20""","""mint""","""eths""","""1000""",null,null
6360372,"""0x91e1c6e17a3f…","""0x646174613a2c…","""0x61fd0d043d51…","""0x61fd0d043d51…",2023-06-18 09:44:30,219600,250000000,0.000055,1,"""{""p"":""erc-20"",…","""erc-20""","""mint""","""eths""","""1000""",null,null
6360489,"""0x0c3e0555c76f…","""0x646174613a2c…","""0x531e2b809625…","""0x531e2b809625…",2023-06-18 09:46:28,220008,250000000,0.000055,1,"""{""p"":""erc-20"",…","""erc-20""","""mint""","""eths""","""1000""",null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
29797541,"""0x6d7a455b610e…","""0x646174613a2c…","""0xd3c90777cd42…","""0xd3c90777cd42…",2024-03-25 01:37:58,306798,25000000,0.000008,1,"""{""p"":""zrc-20"",…","""zrc-20""","""mint""","""sync""","""4""",null,null
29798894,"""0x8107111e20db…","""0x646174613a2c…","""0x04cd82afeda3…","""0x0a88bc5c32b6…",2024-03-25 02:01:56,184878,25000000,0.000005,1,"""{""p"":""layer2-2…","""layer2-20""","""claim""","""$L2""","""1000""",null,null
29799033,"""0xfd3d42ba744c…","""0x646174613a2c…","""0x4280e40231e3…","""0x4280e40231e3…",2024-03-25 02:04:15,146423,25000000,0.000004,1,"""{""p"":""zrc-20"",…","""zrc-20""","""mint""","""sync""","""4""",null,null


In [9]:
del tokens_df

In [10]:
data = inscriptions_parsed.filter(pl.col('p').is_not_null() & (pl.col('p') != ''))

value_counts = data.group_by('p').count().sort('count', descending=True)
top_protocols = value_counts.head(6)['p']

value_counts = data.group_by('tick').count().sort('count', descending=True)
top_ticks = value_counts.head(6)['tick']

value_counts = data.group_by('op').count().sort('count', descending=True)
top_operations = value_counts.head(6)['op']

data = data.with_columns(pl.when(pl.col('p').is_in(top_protocols)).then('p').otherwise(pl.lit('Others')).alias('p'))
data = data.with_columns(pl.when(pl.col('tick').is_in(top_ticks)).then('tick').otherwise(pl.lit('Others')).alias('tick'))
data = data.with_columns(pl.when(pl.col('op').is_in(top_operations)).then('op').otherwise(pl.lit('Others')).alias('op'))

value_counts = data.group_by('tick', 'p', 'op').count().sort('tick', 'p', 'op', descending=True)


value_counts

/var/folders/y_/27jchs_x56gfpv07wkchjxj40000gn/T/ipykernel_1638/317071789.py:3: DeprecationWarning: `count` is deprecated. It has been renamed to `len`.
  value_counts = data.group_by('p').count().sort('count', descending=True)
/var/folders/y_/27jchs_x56gfpv07wkchjxj40000gn/T/ipykernel_1638/317071789.py:6: DeprecationWarning: `count` is deprecated. It has been renamed to `len`.
  value_counts = data.group_by('tick').count().sort('count', descending=True)
/var/folders/y_/27jchs_x56gfpv07wkchjxj40000gn/T/ipykernel_1638/317071789.py:9: DeprecationWarning: `count` is deprecated. It has been renamed to `len`.
  value_counts = data.group_by('op').count().sort('count', descending=True)
/var/folders/y_/27jchs_x56gfpv07wkchjxj40000gn/T/ipykernel_1638/317071789.py:16: DeprecationWarning: `count` is deprecated. It has been renamed to `len`.
  value_counts = data.group_by('tick', 'p', 'op').count().sort('tick', 'p', 'op', descending=True)


tick,p,op,count
str,str,str,u32
"""zkzk""","""zrc-20""","""transfer""",171
"""zkzk""","""zrc-20""","""mint""",1219956
"""zkzk""","""zrc-20""","""deploy""",312
"""zkzk""","""zks-20""","""mint""",1
"""zkzk""","""zks-20""","""deploy""",1
…,…,…,…
"""$L2""","""layer2-20""","""claim""",5651395
"""$L2""","""layer2-20""","""Others""",4
"""$L2""","""Others""","""mint""",21


In [33]:
#data = inscriptions_parsed.filter((pl.col('p')=='zrc-20') & (pl.col('tick')=='sync') & (pl.col('op') == 'transfer'))
data = inscriptions_parsed.filter((pl.col('p')=='era-20') &  (pl.col('op') == 'buy'))

data

block_number,tx_hash,tx_input_data,issuer,receiver,timestamp,gas_used,gas_effective_price,fees,tx_status,decoded_input_data,p,op,tick,amt,tx,price
i64,str,str,str,str,datetime[μs],i64,i64,f64,i64,str,str,str,str,str,str,str
22233365,"""0x4a9c0858e3d4…","""0x646174613a2c…","""0xcd5446c946f4…","""0xf4f837133a7b…",2023-12-23 15:57:49,258436,150000000,0.000039,1,"""{""p"":""era-20"",…","""era-20""","""buy""",null,null,"""57d7eb724a78a2…",null
22891846,"""0x9550ac3a5850…","""0x646174613a2c…","""0x055c851d9d3a…","""0xc84567f12d0e…",2023-12-31 18:54:30,190106,150000000,0.000029,1,"""{""p"":""era-20"",…","""era-20""","""buy""",null,null,"""0x57d7e...695d…",null
22892003,"""0x70a3781ced6c…","""0x646174613a2c…","""0x055c851d9d3a…","""0xc84567f12d0e…",2023-12-31 18:57:13,190918,150000000,0.000029,1,"""{""p"":""era-20"",…","""era-20""","""buy""",null,null,"""0x57d7eb724a78…",null
22894412,"""0x24675d250d17…","""0x646174613a2c…","""0x055c851d9d3a…","""0x055c851d9d3a…",2023-12-31 19:38:43,187463,150000000,0.000028,1,"""{""p"":""era-20"",…","""era-20""","""buy""",null,null,"""0xb7873241572a…",null
22895362,"""0xf5919bbfcbc0…","""0x646174613a2c…","""0x055c851d9d3a…","""0x055c851d9d3a…",2023-12-31 19:55:03,200689,150000000,0.00003,1,"""{""p"":""era-20"",…","""era-20""","""buy""",null,null,"""0x05388d7a8e00…",null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
29753896,"""0x1f3612377013…","""0x646174613a2c…","""0xb92ec3280324…","""0x3894af06d4f9…",2024-03-24 12:44:48,126878,25000000,0.000003,1,"""{""p"":""era-20"",…","""era-20""","""buy""",null,null,"""0xf7324c791e29…",null
29754661,"""0x4c72611e5353…","""0x646174613a2c…","""0xc0d861751bc0…","""0xb92ec3280324…",2024-03-24 12:58:51,205424,25000000,0.000005,1,"""{""p"":""era-20"",…","""era-20""","""buy""",null,null,"""0x4fb85de02857…",null
29760265,"""0x0d6ae45b4c7d…","""0x646174613a2c…","""0x055266b934a8…","""0x28893320e3ec…",2024-03-24 14:40:01,219770,25000000,0.000005,1,"""{""p"":""era-20"",…","""era-20""","""buy""",null,null,"""0xbe1a48aeacc6…",null


In [ ]:
inscriptions_parsed.filter(pl.col('price').is_not_null()).head()

### What are the top deployed inscriptions on zkSync?


In [ ]:
inscriptions_parsed['op'].value_counts().sort('count', descending=True).head(10)

In [ ]:
data = inscriptions_parsed[['p']].to_pandas().copy()
top_protocols = data['p'].value_counts().iloc[:5].index
data.loc[~data['p'].isin(top_protocols), 'p'] = 'Others'

print(data['p'].value_counts(normalize=False))
data = data['p'].value_counts(normalize=True)*100

fig = go.Figure(layout=get_plotly_layout(width=width, height=height))

fig.add_trace(go.Bar(x=data.index, y=data.values,
              marker_color=colors['blue'], textposition='auto', text=data.values.round(2), name='Protocols'))
fig.update_layout(yaxis_title="Percentage",
                  xaxis_title="Protocol", yaxis_ticksuffix="%")

fig.update_traces(
    texttemplate='<b>%{text:,.4}</b>', textfont_size=18)
fig.update_yaxes(range=[0, 100])

fig.write_image(plots_dir+"top-15-protocols-zksync.pdf")

fig.show('png')

In [ ]:
data = inscriptions_parsed[['op']].to_pandas().copy()
top_protocols = data['op'].value_counts().iloc[:5].index
data.loc[~data['op'].isin(top_protocols), 'op'] = 'Others'

print(data['op'].value_counts(normalize=False))
data = data['op'].value_counts(normalize=True)*100

fig = go.Figure(layout=get_plotly_layout(width=width, height=height))

fig.add_trace(go.Bar(x=data.index, y=data.values,
              marker_color=colors['blue'], textposition='auto', text=data.values.round(2), name='Operation'))
fig.update_layout(yaxis_title="Percentage",
                  xaxis_title="Operation", yaxis_ticksuffix="%")

fig.update_traces(
    texttemplate='<b>%{text:,.4}</b>', textfont_size=18)
fig.update_yaxes(range=[0, 100])

fig.write_image(plots_dir+"top-15-operation-zksync.pdf")

fig.show('png')

In [ ]:
data = inscriptions_parsed[['tick']].to_pandas().copy()
top_protocols = data['tick'].value_counts().iloc[:5].index
data.loc[~data['tick'].isin(top_protocols), 'tick'] = 'Others'

print(data['tick'].value_counts(normalize=False))
data = data['tick'].value_counts(normalize=True)*100

fig = go.Figure(layout=get_plotly_layout(width=width, height=height))

fig.add_trace(go.Bar(x=data.index, y=data.values,
              marker_color=colors['blue'], textposition='auto', text=data.values.round(2), name='Protocols'))
fig.update_layout(yaxis_title="Percentage",
                  xaxis_title="Tick", yaxis_ticksuffix="%")

fig.update_traces(
    texttemplate='<b>%{text:,.4}</b>', textfont_size=18)
fig.update_yaxes(range=[0, 100])

fig.write_image(plots_dir+"top-15-tick-zksync.pdf")

fig.show('png')

In [ ]:
inscriptions_parsed.filter(pl.col('tick').eq('bgnt') & pl.col('op').eq('list'))

In [ ]:
inscriptions_parsed.filter(pl.col('p').eq('era-20') & pl.col('op').eq('buy'))

In [ ]:
data = inscriptions_parsed.filter(pl.col('op').eq('deploy')).to_pandas().copy()
top_protocols = data['p'].value_counts().iloc[:5].index
data.loc[~data['p'].isin(top_protocols), 'p'] = 'Others'

print(data['p'].value_counts(normalize=False))
data = data['p'].value_counts(normalize=True)*100


fig = go.Figure(layout=get_plotly_layout(width=width, height=height))

fig.add_trace(go.Bar(x=data.index, y=data.values,
              marker_color=colors['blue'], textposition='auto', text=data.values.round(2), name='Protocols'))
fig.update_layout(yaxis_title="Percentage",
                  xaxis_title="Protocols deployed", yaxis_ticksuffix="%")

fig.update_traces(
    texttemplate='<b>%{text:,.4}</b>', textfont_size=18)
fig.update_yaxes(range=[0, 100])

fig.write_image(plots_dir+"top-15-deployed-protocols-zksync.pdf")

fig.show('png')

In [ ]:
data = inscriptions_parsed.filter(pl.col('op').eq('deploy')).to_pandas().copy()
top_protocols = data['tick'].value_counts().iloc[:5].index
data.loc[~data['tick'].isin(top_protocols), 'tick'] = 'Others'

print(data['tick'].value_counts(normalize=False))
data = data['tick'].value_counts(normalize=True)*100


fig = go.Figure(layout=get_plotly_layout(width=width, height=height))

fig.add_trace(go.Bar(x=data.index, y=data.values,
              marker_color=colors['blue'], textposition='auto', text=data.values.round(2), name='Ticks'))
fig.update_layout(yaxis_title="Percentage",
                  xaxis_title="Ticks deployed", yaxis_ticksuffix="%")

fig.update_traces(
    texttemplate='<b>%{text:,.4}</b>', textfont_size=18)
fig.update_yaxes(range=[0, 100])

fig.write_image(plots_dir+"ticks-zksync.pdf")

fig.show('png')

In [ ]:
# Loading all txs inscriptions dataframe from a file
all_txs_df = pl.scan_parquet(
    data_dir+'inscriptions_all_txs_df.parquet').collect(streaming=True)
print("There are {} txs in our dataset.".format(
    all_txs_df.shape[0]))
all_txs_df.head()

In [ ]:
print("There are {} unique transactions in our dataset.".format(
    all_txs_df['tx_hash'].n_unique()))
print("There are {} unique blocks in our dataset.".format(
    all_txs_df['block_number'].n_unique()))
print("The average number of txs per block is {} txs.".format(round(
    all_txs_df['tx_hash'].n_unique()/all_txs_df['block_number'].n_unique(), 2)))

print("The minimum and max number of txs per block are: {} and {}.".format(
    all_txs_df['block_number'].min(), all_txs_df['block_number'].max()))

print("The minimum timestamp is {} and the maximum timestamp is {}.".format(
    all_txs_df['timestamp'].min(), all_txs_df['timestamp'].max()))

In [ ]:
# Number of inscriptions in comparison to the total number of transactions per day on zkSync Era

all_txs_per_day = (all_txs_df[['block_number', 'timestamp']]
                   .sort('timestamp').group_by_dynamic("timestamp", every="1d")
                   .agg(pl.len()))

inscriptions_txs_per_day = (all_txs_df.filter(pl.col('is_inscription').eq(True))[['block_number', 'timestamp']]
                            .sort('timestamp')
                            .group_by_dynamic("timestamp", every="1d")
                            .agg(pl.len()))


fig = go.Figure(layout=get_plotly_layout(width=width, height=height))
fig.add_trace(go.Scatter(
    x=all_txs_per_day['timestamp'], y=all_txs_per_day['len'], line=dict(color=colors['blue'], width=3, dash='solid'), mode='lines', name='Transactions'))
fig.add_trace(go.Scatter(
    x=inscriptions_txs_per_day['timestamp'], y=inscriptions_txs_per_day['len'], line=dict(color=colors['red'], width=3, dash='solid'),  mode='lines', name='Inscripted transactions'))
# fig.add_trace(go.Scatter(
#     x=self_txs_per_day['timestamp'], y=self_txs_per_day['len'], line=dict(color=colors['green'], width=3, dash='solid'),  mode='lines', name='Self-transfer transactions'))

fig.update_layout(yaxis_title="Number of transactions",
                  xaxis_title="Date", legend=dict(xanchor='center',
                                                  x=0.5, y=1.02, orientation='h'))

fig.write_image(plots_dir+'fraction-of-inscriptions-zksync.pdf')
fig.show('png')

In [ ]:
# Distribution of the percentage of inscriptions in comparison to the total number of transactions per week on zkSync

all_txs_per_day = (all_txs_df[['block_number', 'timestamp']]
                   .sort('timestamp').group_by_dynamic("timestamp", every="6h")
                   .agg(pl.len()))

inscriptions_txs_per_day = (all_txs_df.filter(pl.col('is_inscription').eq(True))[['block_number', 'timestamp']]
                            .sort('timestamp')
                            .group_by_dynamic("timestamp", every="6h")
                            .agg(pl.len()))

data = inscriptions_txs_per_day.join(all_txs_per_day, on='timestamp', how='left', suffix='_all_txs').with_columns(
    pl.col('len').truediv(pl.col('len_all_txs')).alias('fraction'))


fig = go.Figure(layout=get_plotly_layout(width=width, height=height))
fig.add_trace(go.Scatter(
    x=data['timestamp'], y=data['fraction']*100, line=dict(color=colors['blue'], width=3, dash='solid'), mode='lines', name='Transactions'))

fig.update_xaxes(range=['2023-11-09', '2024-03-24'])

fig.update_yaxes(range=[0, 100])

fig.update_layout(yaxis_title="Percentage",
                  xaxis_title="Date",  yaxis_ticksuffix="%", legend=dict(xanchor='center',
                                                                         x=0.5, y=1.02, orientation='h'))

fig.write_image(plots_dir+'fraction-of-inscriptions-zksync.pdf')
fig.show('png')

In [34]:
inscription_buysell = inscriptions_parsed.filter((pl.col('op')=='buy') | (pl.col('op')=='sell'))

In [35]:
inscription_buysell.unique('p')
#only zrc-20 and era-20

block_number,tx_hash,tx_input_data,issuer,receiver,timestamp,gas_used,gas_effective_price,fees,tx_status,decoded_input_data,p,op,tick,amt,tx,price
i64,str,str,str,str,datetime[μs],i64,i64,f64,i64,str,str,str,str,str,str,str
22233365,"""0x4a9c0858e3d4…","""0x646174613a2c…","""0xcd5446c946f4…","""0xf4f837133a7b…",2023-12-23 15:57:49,258436,150000000,0.000039,1,"""{""p"":""era-20"",…","""era-20""","""buy""",null,null,"""57d7eb724a78a2…",null
21964532,"""0x115725014c9b…","""0x646174613a2c…","""0xacac7716be22…","""0xaa97f900599d…",2023-12-20 10:39:45,653341,190000000,0.000124,1,"""{""p"":""zrc-20"",…","""zrc-20""","""sell""",null,null,null,null


In [36]:
merged_df = inscription_buysell.join(inscriptions_parsed, how='inner', left_on='tx', right_on = 'tx_hash' )

print("There are {} merged buy and sell.".format(merged_df.shape[0]))
print("Out of total {} of buy and sell.".format(inscription_buysell.shape[0]))
merged_df.head()


There are 1204 merged buy and sell.
Out of total 1309 of buy and sell.


block_number,tx_hash,tx_input_data,issuer,receiver,timestamp,gas_used,gas_effective_price,fees,tx_status,decoded_input_data,p,op,tick,amt,tx,price,block_number_right,tx_input_data_right,issuer_right,receiver_right,timestamp_right,gas_used_right,gas_effective_price_right,fees_right,tx_status_right,decoded_input_data_right,p_right,op_right,tick_right,amt_right,tx_right,price_right
i64,str,str,str,str,datetime[μs],i64,i64,f64,i64,str,str,str,str,str,str,str,i64,str,str,str,datetime[μs],i64,i64,f64,i64,str,str,str,str,str,str,str
24378005,"""0x1babe31683ef…","""0x646174613a2c…","""0x0f63ae83d785…","""0xcd5446c946f4…",2024-01-18 18:14:06,627369,100000000,0.000063,1,"""{""p"":""era-20"",…","""era-20""","""buy""",null,null,"""0xccca8ddf683d…",null,22219206,"""0x646174613a2c…","""0xcd5446c946f4…","""0xcd5446c946f4…",2023-12-23 11:49:37,204594,150000000,0.000031,1,"""{""p"":""era-20"",…","""era-20""","""list""","""bgnt""","""5""",null,"""10000000000000…"
22892003,"""0x70a3781ced6c…","""0x646174613a2c…","""0x055c851d9d3a…","""0xc84567f12d0e…",2023-12-31 18:57:13,190918,150000000,0.000029,1,"""{""p"":""era-20"",…","""era-20""","""buy""",null,null,"""0x57d7eb724a78…",null,22233218,"""0x646174613a2c…","""0xf4f837133a7b…","""0xf4f837133a7b…",2023-12-23 15:55:16,271386,150000000,0.000041,1,"""{""p"":""era-20"",…","""era-20""","""list""","""bgnt""","""5""",null,"""10000000000000…"
22901583,"""0x20db3f736cb4…","""0x646174613a2c…","""0x055c851d9d3a…","""0xf4f837133a7b…",2023-12-31 21:41:35,211201,150000000,0.000032,1,"""{""p"":""era-20"",…","""era-20""","""buy""",null,null,"""0x57d7eb724a78…",null,22233218,"""0x646174613a2c…","""0xf4f837133a7b…","""0xf4f837133a7b…",2023-12-23 15:55:16,271386,150000000,0.000041,1,"""{""p"":""era-20"",…","""era-20""","""list""","""bgnt""","""5""",null,"""10000000000000…"
22895362,"""0xf5919bbfcbc0…","""0x646174613a2c…","""0x055c851d9d3a…","""0x055c851d9d3a…",2023-12-31 19:55:03,200689,150000000,0.00003,1,"""{""p"":""era-20"",…","""era-20""","""buy""",null,null,"""0x05388d7a8e00…",null,22890920,"""0x646174613a2c…","""0x055c851d9d3a…","""0x055c851d9d3a…",2023-12-31 18:38:32,196051,150000000,0.000029,1,"""{""p"":""era-20"",…","""era-20""","""list""","""bgnt""","""5""",null,"""0.000000000000…"
22895909,"""0x5a0412b4eaa1…","""0x646174613a2c…","""0x055c851d9d3a…","""0x055c851d9d3a…",2023-12-31 20:04:28,198759,150000000,0.00003,1,"""{""p"":""era-20"",…","""era-20""","""buy""",null,null,"""0x05388d7a8e00…",null,22890920,"""0x646174613a2c…","""0x055c851d9d3a…","""0x055c851d9d3a…",2023-12-31 18:38:32,196051,150000000,0.000029,1,"""{""p"":""era-20"",…","""era-20""","""list""","""bgnt""","""5""",null,"""0.000000000000…"


In [37]:
merged_df = merged_df.with_columns(pl.col('amt_right').cast(pl.Int32))

In [38]:
merged_df = merged_df.with_columns(pl.col('price_right').cast(pl.Float32))
merged_df

block_number,tx_hash,tx_input_data,issuer,receiver,timestamp,gas_used,gas_effective_price,fees,tx_status,decoded_input_data,p,op,tick,amt,tx,price,block_number_right,tx_input_data_right,issuer_right,receiver_right,timestamp_right,gas_used_right,gas_effective_price_right,fees_right,tx_status_right,decoded_input_data_right,p_right,op_right,tick_right,amt_right,tx_right,price_right
i64,str,str,str,str,datetime[μs],i64,i64,f64,i64,str,str,str,str,str,str,str,i64,str,str,str,datetime[μs],i64,i64,f64,i64,str,str,str,str,i32,str,f32
24378005,"""0x1babe31683ef…","""0x646174613a2c…","""0x0f63ae83d785…","""0xcd5446c946f4…",2024-01-18 18:14:06,627369,100000000,0.000063,1,"""{""p"":""era-20"",…","""era-20""","""buy""",null,null,"""0xccca8ddf683d…",null,22219206,"""0x646174613a2c…","""0xcd5446c946f4…","""0xcd5446c946f4…",2023-12-23 11:49:37,204594,150000000,0.000031,1,"""{""p"":""era-20"",…","""era-20""","""list""","""bgnt""",5,null,1.0000e13
22892003,"""0x70a3781ced6c…","""0x646174613a2c…","""0x055c851d9d3a…","""0xc84567f12d0e…",2023-12-31 18:57:13,190918,150000000,0.000029,1,"""{""p"":""era-20"",…","""era-20""","""buy""",null,null,"""0x57d7eb724a78…",null,22233218,"""0x646174613a2c…","""0xf4f837133a7b…","""0xf4f837133a7b…",2023-12-23 15:55:16,271386,150000000,0.000041,1,"""{""p"":""era-20"",…","""era-20""","""list""","""bgnt""",5,null,1.0000e13
22901583,"""0x20db3f736cb4…","""0x646174613a2c…","""0x055c851d9d3a…","""0xf4f837133a7b…",2023-12-31 21:41:35,211201,150000000,0.000032,1,"""{""p"":""era-20"",…","""era-20""","""buy""",null,null,"""0x57d7eb724a78…",null,22233218,"""0x646174613a2c…","""0xf4f837133a7b…","""0xf4f837133a7b…",2023-12-23 15:55:16,271386,150000000,0.000041,1,"""{""p"":""era-20"",…","""era-20""","""list""","""bgnt""",5,null,1.0000e13
22895362,"""0xf5919bbfcbc0…","""0x646174613a2c…","""0x055c851d9d3a…","""0x055c851d9d3a…",2023-12-31 19:55:03,200689,150000000,0.00003,1,"""{""p"":""era-20"",…","""era-20""","""buy""",null,null,"""0x05388d7a8e00…",null,22890920,"""0x646174613a2c…","""0x055c851d9d3a…","""0x055c851d9d3a…",2023-12-31 18:38:32,196051,150000000,0.000029,1,"""{""p"":""era-20"",…","""era-20""","""list""","""bgnt""",5,null,1.0000e-18
22895909,"""0x5a0412b4eaa1…","""0x646174613a2c…","""0x055c851d9d3a…","""0x055c851d9d3a…",2023-12-31 20:04:28,198759,150000000,0.00003,1,"""{""p"":""era-20"",…","""era-20""","""buy""",null,null,"""0x05388d7a8e00…",null,22890920,"""0x646174613a2c…","""0x055c851d9d3a…","""0x055c851d9d3a…",2023-12-31 18:38:32,196051,150000000,0.000029,1,"""{""p"":""era-20"",…","""era-20""","""list""","""bgnt""",5,null,1.0000e-18
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
29693762,"""0x33d9a68d8920…","""0x646174613a2c…","""0xc0d861751bc0…","""0x22bfcf932947…",2024-03-23 18:58:21,222952,25000000,0.000006,1,"""{""p"":""era-20"",…","""era-20""","""buy""",null,null,"""0xe1c47f3c80b2…",null,29644847,"""0x646174613a2c…","""0x22bfcf932947…","""0x22bfcf932947…",2024-03-23 04:20:09,162187,25000000,0.000004,1,"""{""p"":""era-20"",…","""era-20""","""list""","""bgnt""",500,null,5.0000e14
29788740,"""0xa4d7ecad2649…","""0x646174613a2c…","""0x556940aa64a0…","""0xe9c298cc5697…",2024-03-24 23:04:18,211897,25000000,0.000005,1,"""{""p"":""era-20"",…","""era-20""","""buy""",null,null,"""0x4099c9da7304…",null,29670056,"""0x646174613a2c…","""0xe9c298cc5697…","""0xe9c298cc5697…",2024-03-23 11:53:05,113640,25000000,0.000003,1,"""{""p"":""era-20"",…","""era-20""","""list""","""bgnt""",10,null,1.4000e14
29753896,"""0x1f3612377013…","""0x646174613a2c…","""0xb92ec3280324…","""0x3894af06d4f9…",2024-03-24 12:44:48,126878,25000000,0.000003,1,"""{""p"":""era-20"",…","""era-20""","""buy""",null,null,"""0xf7324c791e29…",null,29753808,"""0x646174613a2c…","""0x3894af06d4f9…","""0x3894af06d4f9…",2024-03-24 12:43:20,162869,25000000,0.000004,1,"""{""p"":""era-20"",…","""era-20""","""list""","""bgnt""",15,null,6.0000e12


In [39]:
merged_df = merged_df.with_columns((pl.col('price_right')/pl.col('amt_right')).alias('price_1token'))
merged_df

block_number,tx_hash,tx_input_data,issuer,receiver,timestamp,gas_used,gas_effective_price,fees,tx_status,decoded_input_data,p,op,tick,amt,tx,price,block_number_right,tx_input_data_right,issuer_right,receiver_right,timestamp_right,gas_used_right,gas_effective_price_right,fees_right,tx_status_right,decoded_input_data_right,p_right,op_right,tick_right,amt_right,tx_right,price_right,price_1token
i64,str,str,str,str,datetime[μs],i64,i64,f64,i64,str,str,str,str,str,str,str,i64,str,str,str,datetime[μs],i64,i64,f64,i64,str,str,str,str,i32,str,f32,f64
24378005,"""0x1babe31683ef…","""0x646174613a2c…","""0x0f63ae83d785…","""0xcd5446c946f4…",2024-01-18 18:14:06,627369,100000000,0.000063,1,"""{""p"":""era-20"",…","""era-20""","""buy""",null,null,"""0xccca8ddf683d…",null,22219206,"""0x646174613a2c…","""0xcd5446c946f4…","""0xcd5446c946f4…",2023-12-23 11:49:37,204594,150000000,0.000031,1,"""{""p"":""era-20"",…","""era-20""","""list""","""bgnt""",5,null,1.0000e13,2.0000e12
22892003,"""0x70a3781ced6c…","""0x646174613a2c…","""0x055c851d9d3a…","""0xc84567f12d0e…",2023-12-31 18:57:13,190918,150000000,0.000029,1,"""{""p"":""era-20"",…","""era-20""","""buy""",null,null,"""0x57d7eb724a78…",null,22233218,"""0x646174613a2c…","""0xf4f837133a7b…","""0xf4f837133a7b…",2023-12-23 15:55:16,271386,150000000,0.000041,1,"""{""p"":""era-20"",…","""era-20""","""list""","""bgnt""",5,null,1.0000e13,2.0000e12
22901583,"""0x20db3f736cb4…","""0x646174613a2c…","""0x055c851d9d3a…","""0xf4f837133a7b…",2023-12-31 21:41:35,211201,150000000,0.000032,1,"""{""p"":""era-20"",…","""era-20""","""buy""",null,null,"""0x57d7eb724a78…",null,22233218,"""0x646174613a2c…","""0xf4f837133a7b…","""0xf4f837133a7b…",2023-12-23 15:55:16,271386,150000000,0.000041,1,"""{""p"":""era-20"",…","""era-20""","""list""","""bgnt""",5,null,1.0000e13,2.0000e12
22895362,"""0xf5919bbfcbc0…","""0x646174613a2c…","""0x055c851d9d3a…","""0x055c851d9d3a…",2023-12-31 19:55:03,200689,150000000,0.00003,1,"""{""p"":""era-20"",…","""era-20""","""buy""",null,null,"""0x05388d7a8e00…",null,22890920,"""0x646174613a2c…","""0x055c851d9d3a…","""0x055c851d9d3a…",2023-12-31 18:38:32,196051,150000000,0.000029,1,"""{""p"":""era-20"",…","""era-20""","""list""","""bgnt""",5,null,1.0000e-18,2.0000e-19
22895909,"""0x5a0412b4eaa1…","""0x646174613a2c…","""0x055c851d9d3a…","""0x055c851d9d3a…",2023-12-31 20:04:28,198759,150000000,0.00003,1,"""{""p"":""era-20"",…","""era-20""","""buy""",null,null,"""0x05388d7a8e00…",null,22890920,"""0x646174613a2c…","""0x055c851d9d3a…","""0x055c851d9d3a…",2023-12-31 18:38:32,196051,150000000,0.000029,1,"""{""p"":""era-20"",…","""era-20""","""list""","""bgnt""",5,null,1.0000e-18,2.0000e-19
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
29693762,"""0x33d9a68d8920…","""0x646174613a2c…","""0xc0d861751bc0…","""0x22bfcf932947…",2024-03-23 18:58:21,222952,25000000,0.000006,1,"""{""p"":""era-20"",…","""era-20""","""buy""",null,null,"""0xe1c47f3c80b2…",null,29644847,"""0x646174613a2c…","""0x22bfcf932947…","""0x22bfcf932947…",2024-03-23 04:20:09,162187,25000000,0.000004,1,"""{""p"":""era-20"",…","""era-20""","""list""","""bgnt""",500,null,5.0000e14,1.0000e12
29788740,"""0xa4d7ecad2649…","""0x646174613a2c…","""0x556940aa64a0…","""0xe9c298cc5697…",2024-03-24 23:04:18,211897,25000000,0.000005,1,"""{""p"":""era-20"",…","""era-20""","""buy""",null,null,"""0x4099c9da7304…",null,29670056,"""0x646174613a2c…","""0xe9c298cc5697…","""0xe9c298cc5697…",2024-03-23 11:53:05,113640,25000000,0.000003,1,"""{""p"":""era-20"",…","""era-20""","""list""","""bgnt""",10,null,1.4000e14,1.4000e13
29753896,"""0x1f3612377013…","""0x646174613a2c…","""0xb92ec3280324…","""0x3894af06d4f9…",2024-03-24 12:44:48,126878,25000000,0.000003,1,"""{""p"":""era-20"",…","""era-20""","""buy""",null,null,"""0xf7324c791e29…",null,29753808,"""0x646174613a2c…","""0x3894af06d4f9…","""0x3894af06d4f9…",2024-03-24 12:43:20,162869,25000000,0.000004,1,"""{""p"":""era-20"",…","""era-20""","""list""","""bgnt""",15,null,6.0000e12,4.0000e11


In [19]:
merged_df['tx_hash'][3]

'0xf5919bbfcbc009189fda33c23f1adf819498df1c7b065cb7e5b81f4fefd9daaa'

In [40]:
value_counts = merged_df.group_by('tick', 'tick_right', 'p', 'p_right', 'op', 'op_right').count().sort('tick', 'p', 'op', descending=True)
value_counts

/var/folders/y_/27jchs_x56gfpv07wkchjxj40000gn/T/ipykernel_1638/1756438383.py:1: DeprecationWarning: `count` is deprecated. It has been renamed to `len`.
  value_counts = merged_df.group_by('tick', 'tick_right', 'p', 'p_right', 'op', 'op_right').count().sort('tick', 'p', 'op', descending=True)


tick,tick_right,p,p_right,op,op_right,count
str,str,str,str,str,str,u32
null,"""bgnt""","""era-20""","""era-20""","""buy""","""list""",1148
null,"""meme""","""era-20""","""era-20""","""buy""","""list""",18
null,"""era20""","""era-20""","""era-20""","""buy""","""list""",38


In [41]:
data = merged_df.filter((pl.col('tick_right')=='bgnt'))


In [42]:
data = data.with_columns((pl.col('price_1token')/1e18).alias('price_1token'))
data

block_number,tx_hash,tx_input_data,issuer,receiver,timestamp,gas_used,gas_effective_price,fees,tx_status,decoded_input_data,p,op,tick,amt,tx,price,block_number_right,tx_input_data_right,issuer_right,receiver_right,timestamp_right,gas_used_right,gas_effective_price_right,fees_right,tx_status_right,decoded_input_data_right,p_right,op_right,tick_right,amt_right,tx_right,price_right,price_1token
i64,str,str,str,str,datetime[μs],i64,i64,f64,i64,str,str,str,str,str,str,str,i64,str,str,str,datetime[μs],i64,i64,f64,i64,str,str,str,str,i32,str,f32,f64
24378005,"""0x1babe31683ef…","""0x646174613a2c…","""0x0f63ae83d785…","""0xcd5446c946f4…",2024-01-18 18:14:06,627369,100000000,0.000063,1,"""{""p"":""era-20"",…","""era-20""","""buy""",null,null,"""0xccca8ddf683d…",null,22219206,"""0x646174613a2c…","""0xcd5446c946f4…","""0xcd5446c946f4…",2023-12-23 11:49:37,204594,150000000,0.000031,1,"""{""p"":""era-20"",…","""era-20""","""list""","""bgnt""",5,null,1.0000e13,0.000002
22892003,"""0x70a3781ced6c…","""0x646174613a2c…","""0x055c851d9d3a…","""0xc84567f12d0e…",2023-12-31 18:57:13,190918,150000000,0.000029,1,"""{""p"":""era-20"",…","""era-20""","""buy""",null,null,"""0x57d7eb724a78…",null,22233218,"""0x646174613a2c…","""0xf4f837133a7b…","""0xf4f837133a7b…",2023-12-23 15:55:16,271386,150000000,0.000041,1,"""{""p"":""era-20"",…","""era-20""","""list""","""bgnt""",5,null,1.0000e13,0.000002
22901583,"""0x20db3f736cb4…","""0x646174613a2c…","""0x055c851d9d3a…","""0xf4f837133a7b…",2023-12-31 21:41:35,211201,150000000,0.000032,1,"""{""p"":""era-20"",…","""era-20""","""buy""",null,null,"""0x57d7eb724a78…",null,22233218,"""0x646174613a2c…","""0xf4f837133a7b…","""0xf4f837133a7b…",2023-12-23 15:55:16,271386,150000000,0.000041,1,"""{""p"":""era-20"",…","""era-20""","""list""","""bgnt""",5,null,1.0000e13,0.000002
22895362,"""0xf5919bbfcbc0…","""0x646174613a2c…","""0x055c851d9d3a…","""0x055c851d9d3a…",2023-12-31 19:55:03,200689,150000000,0.00003,1,"""{""p"":""era-20"",…","""era-20""","""buy""",null,null,"""0x05388d7a8e00…",null,22890920,"""0x646174613a2c…","""0x055c851d9d3a…","""0x055c851d9d3a…",2023-12-31 18:38:32,196051,150000000,0.000029,1,"""{""p"":""era-20"",…","""era-20""","""list""","""bgnt""",5,null,1.0000e-18,2.0000e-37
22895909,"""0x5a0412b4eaa1…","""0x646174613a2c…","""0x055c851d9d3a…","""0x055c851d9d3a…",2023-12-31 20:04:28,198759,150000000,0.00003,1,"""{""p"":""era-20"",…","""era-20""","""buy""",null,null,"""0x05388d7a8e00…",null,22890920,"""0x646174613a2c…","""0x055c851d9d3a…","""0x055c851d9d3a…",2023-12-31 18:38:32,196051,150000000,0.000029,1,"""{""p"":""era-20"",…","""era-20""","""list""","""bgnt""",5,null,1.0000e-18,2.0000e-37
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
29693762,"""0x33d9a68d8920…","""0x646174613a2c…","""0xc0d861751bc0…","""0x22bfcf932947…",2024-03-23 18:58:21,222952,25000000,0.000006,1,"""{""p"":""era-20"",…","""era-20""","""buy""",null,null,"""0xe1c47f3c80b2…",null,29644847,"""0x646174613a2c…","""0x22bfcf932947…","""0x22bfcf932947…",2024-03-23 04:20:09,162187,25000000,0.000004,1,"""{""p"":""era-20"",…","""era-20""","""list""","""bgnt""",500,null,5.0000e14,1.0000e-6
29788740,"""0xa4d7ecad2649…","""0x646174613a2c…","""0x556940aa64a0…","""0xe9c298cc5697…",2024-03-24 23:04:18,211897,25000000,0.000005,1,"""{""p"":""era-20"",…","""era-20""","""buy""",null,null,"""0x4099c9da7304…",null,29670056,"""0x646174613a2c…","""0xe9c298cc5697…","""0xe9c298cc5697…",2024-03-23 11:53:05,113640,25000000,0.000003,1,"""{""p"":""era-20"",…","""era-20""","""list""","""bgnt""",10,null,1.4000e14,0.000014
29753896,"""0x1f3612377013…","""0x646174613a2c…","""0xb92ec3280324…","""0x3894af06d4f9…",2024-03-24 12:44:48,126878,25000000,0.000003,1,"""{""p"":""era-20"",…","""era-20""","""buy""",null,null,"""0xf7324c791e29…",null,29753808,"""0x646174613a2c…","""0x3894af06d4f9…","""0x3894af06d4f9…",2024-03-24 12:43:20,162869,25000000,0.000004,1,"""{""p"":""era-20"",…","""era-20""","""list""","""bgnt""",15,null,6.0000e12,4.0000e-7


In [43]:
#data_f = data.filter((pl.col('price_1token')>0.000001) & (pl.col('price_1token').is_not_nan()))
#data_f

data_f = data

In [57]:
data_f['tx_hash'][5]

'0xd1e41a2cf98587102209d8a92988e5275810e8641693d113b9621f69cdfc32ef'

In [45]:
data_f = data_f.with_columns((pl.col('price_1token')*4000).alias('price_1token_USD'))


data_f = data.filter((pl.col('price_1token')>0.000001) & (pl.col('price_1token').is_not_nan()))


In [184]:
data_f['price_1token_USD'].describe()

statistic,value
str,f64
"""count""",1148.0
"""null_count""",0.0
"""mean""",NaN
"""std""",NaN
"""min""",0.0
"""25%""",0.002
"""50%""",0.0324
"""75%""",0.054936
"""max""",NaN


In [118]:
#fig = go.Figure()
fig = go.Figure(layout=layout)


temp_df = data_f.filter((pl.col('price_1token_USD')<1))

fig.add_trace(go.Scatter(x=temp_df['timestamp'], y=temp_df['price_1token_USD'], mode='markers', name='Price'))

# Update layout
fig.update_layout(xaxis_title='Transaction Timestamp ',
                  yaxis_title='Price (USD)')

fig.write_image(plots_dir+"buybgnt_blue.pdf",
                width=1540, height=380, scale=1)

fig.show()

#1) number of sales thorouhg time
#2 )box plot through time -> seaborn
#3) top receiveers, top issuers - 800
#4) check how did mint vs you did list
#5) gas fee ->

#add to conculasions - gas comparison with BTC and ETH -> that memecoins at this price are only feasible at L2s
#profibaikliy of insction on btc and eth vs l2
#yahooo fiuannce pytthon api

In [ ]:

buy_bgnt = data_f.with_columns((pl.col('timestamp').cast(pl.Date)).alias('date'))
buy_bgnt = buy_bgnt.with_columns((pl.col('price_1token_USD')).alias('value_USD'))
buy_bgnt = buy_bgnt.filter((pl.col('value_USD')<1))
data = buy_bgnt.group_by('date').agg(pl.col("value_USD").sum())
data = data.sort(pl.col("date"))
data



In [153]:
fig = go.Figure(layout=layout)


#data = transfer_sync.group_by('date').agg(pl.col("value_USD").sum())
#data = data.sort(pl.col("date"))
#fig.add_trace(go.Scatter(x=data['date'], y=data['value_USD'], line=dict(
#    color=colors['light_purple'], width=4), mode='lines', name='Total Volume'))

buy_bgnt = buy_bgnt.filter((pl.col('value_USD')>1e-10))

data = buy_bgnt.group_by('date').agg(pl.col("value_USD").median())
data = data.sort(pl.col("date"))
fig.add_trace(go.Scatter(x=data['date'], y=data['value_USD'], line=dict(
    color=colors['brown'], width=2), mode='lines', name='Median Price'))

data = buy_bgnt
data = data.sort(pl.col("date"))
fig.add_trace(go.Box(x=data['date'], y=data['value_USD'],
              name='Transaction Price', marker_color=colors['green'], line=dict(width=1.5), whiskerwidth=0.5, fillcolor=colors['white']))

# Update layout
fig.update_layout(xaxis_title='Transaction Date',
                  yaxis_title='Price (USD)', legend=dict(xanchor='center',
                                                  x=0.5, y=1.02, orientation='h'))

#fig.update_yaxes(type='log', range=[-10, 1], tickvals=[1, 0.1, 0.01, 0.001, 0.0001])
#fig.update_yaxes(type='log', range=[-11, 2])

fig.write_image(plots_dir+"buybgnt.pdf",
                width=1540, height=380, scale=1)
fig.show()

In [204]:

df = buy_bgnt.select(pl.std("value_USD"))
df["value_USD"]

#df = buy_bgnt.select(pl.median("value_USD"))
#df["value_USD"]

#df = buy_bgnt.select(pl.mean("value_USD"))
#df["value_USD"]

#df = buy_bgnt.select(pl.min("value_USD"))
#df["value_USD"]

#df = buy_bgnt.select(pl.max("value_USD"))
#df["value_USD"]

value_USD
f64
0.078061


In [58]:
inscriptions_transfer = inscriptions_parsed.filter((pl.col('op')=='transfer'))
inscriptions_transfer

In [64]:
inscriptions_transfer.unique('tick')
value_counts = inscriptions_transfer.group_by('tick', 'p').count().sort('count', descending=True)
value_counts


/var/folders/y_/27jchs_x56gfpv07wkchjxj40000gn/T/ipykernel_2844/4291665390.py:2: DeprecationWarning:

`count` is deprecated. It has been renamed to `len`.



tick,p,count
str,str,u32
"""sync""","""zrc-20""",477
"""bgnt""","""era-20""",228
"""zkzk""","""zrc-20""",171
"""zkss""","""zrc-20""",98
"""SHIB""","""zrc-20""",77
…,…,…
"""meme""","""era-20""",1
"""MNTL""","""mrc-20""",1
"""MOMO""","""orc-20""",1


In [60]:
transfer_sync = inscriptions_parsed.filter((pl.col('op')=='transfer') & (pl.col('p')=='zrc-20') & (pl.col('tick')=='sync') )
transfer_sync


block_number,tx_hash,tx_input_data,issuer,receiver,timestamp,gas_used,gas_effective_price,fees,tx_status,decoded_input_data,p,op,tick,amt,tx,price
i64,str,str,str,str,datetime[μs],i64,i64,f64,i64,str,str,str,str,str,str,str
22048559,"""0xcad1a24e5e97…","""0x646174613a2c…","""0x4749ab4b7b47…","""0xb14f884adb66…",2023-12-21 10:40:14,248010,190000000,0.000047,1,"""{""p"":""zrc-20"",…","""zrc-20""","""transfer""","""sync""","""480""",null,null
22048747,"""0x9ce1f100c72a…","""0x646174613a2c…","""0x0a167d46ddf4…","""0xd7fdc1c61b53…",2023-12-21 10:43:28,249245,190000000,0.000047,1,"""{""p"":""zrc-20"",…","""zrc-20""","""transfer""","""sync""","""4""",null,null
22049338,"""0x5ddac4627c80…","""0x646174613a2c…","""0x484da9bda5e0…","""0x484da9bda5e0…",2023-12-21 10:53:43,253708,190000000,0.000048,1,"""{""p"":""zrc-20"",…","""zrc-20""","""transfer""","""sync""","""1""",null,null
22049457,"""0x103b5dffcc19…","""0x646174613a2c…","""0x35839602105d…","""0x484da9bda5e0…",2023-12-21 10:55:48,254897,190000000,0.000048,1,"""{""p"":""zrc-20"",…","""zrc-20""","""transfer""","""sync""","""1""",null,null
22049549,"""0xd4218e38f169…","""0x646174613a2c…","""0x3523f08e37d7…","""0x0dce56a50b39…",2023-12-21 10:57:22,254908,190000000,0.000048,1,"""{""p"":""zrc-20"",…","""zrc-20""","""transfer""","""sync""","""2952""",null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
24269907,"""0xbe1c2d2a167a…","""0x646174613a2c…","""0xde404aaa7437…","""0x5ba5cf528bae…",2024-01-17 11:06:12,417820,100000000,0.000042,1,"""{""p"":""zrc-20"",…","""zrc-20""","""transfer""","""sync""","""24""",null,null
24269941,"""0xee9659c534e9…","""0x646174613a2c…","""0x508703228fed…","""0x5ba5cf528bae…",2024-01-17 11:06:48,417835,100000000,0.000042,1,"""{""p"":""zrc-20"",…","""zrc-20""","""transfer""","""sync""","""124""",null,null
25022829,"""0x38a333db012c…","""0x646174613a2c…","""0x9be02d4c7987…","""0x3551bab7c6c8…",2024-01-26 14:18:53,275107,100000000,0.000028,1,"""{""p"":""zrc-20"",…","""zrc-20""","""transfer""","""sync""","""980""",null,null


In [61]:
import os
import requests as re
from web3 import Web3
import json
import time

# This code connects to our zkSync Era node
eth_node = 'https://mainnet.era.zksync.io'

adapter = re.adapters.HTTPAdapter(pool_connections=20, pool_maxsize=20)
session = re.Session()
session.mount('http://', adapter)
session.mount('https://', adapter)

w3 = Web3(Web3.HTTPProvider(eth_node, session=session,
          request_kwargs={'timeout': 60}))

print("Is connected to Ethereum node: ", w3.is_connected())
print("The most recent block is: ", w3.eth.block_number)

/Users/kris/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning:

urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020



Is connected to Ethereum node:  True
The most recent block is:  31573544


In [90]:
tx_hash = transfer_sync['tx_hash'][10]
tx_data = w3.eth.get_transaction_receipt(tx_hash)

value = int(tx_data['logs'][0]['data'].hex(), 16)/1e18
value

0.000140241888

In [62]:
transfer_sync.with_columns(
    value = 0
)

block_number,tx_hash,tx_input_data,issuer,receiver,timestamp,gas_used,gas_effective_price,fees,tx_status,decoded_input_data,p,op,tick,amt,tx,price,value
i64,str,str,str,str,datetime[μs],i64,i64,f64,i64,str,str,str,str,str,str,str,i32
22048559,"""0xcad1a24e5e97…","""0x646174613a2c…","""0x4749ab4b7b47…","""0xb14f884adb66…",2023-12-21 10:40:14,248010,190000000,0.000047,1,"""{""p"":""zrc-20"",…","""zrc-20""","""transfer""","""sync""","""480""",null,null,0
22048747,"""0x9ce1f100c72a…","""0x646174613a2c…","""0x0a167d46ddf4…","""0xd7fdc1c61b53…",2023-12-21 10:43:28,249245,190000000,0.000047,1,"""{""p"":""zrc-20"",…","""zrc-20""","""transfer""","""sync""","""4""",null,null,0
22049338,"""0x5ddac4627c80…","""0x646174613a2c…","""0x484da9bda5e0…","""0x484da9bda5e0…",2023-12-21 10:53:43,253708,190000000,0.000048,1,"""{""p"":""zrc-20"",…","""zrc-20""","""transfer""","""sync""","""1""",null,null,0
22049457,"""0x103b5dffcc19…","""0x646174613a2c…","""0x35839602105d…","""0x484da9bda5e0…",2023-12-21 10:55:48,254897,190000000,0.000048,1,"""{""p"":""zrc-20"",…","""zrc-20""","""transfer""","""sync""","""1""",null,null,0
22049549,"""0xd4218e38f169…","""0x646174613a2c…","""0x3523f08e37d7…","""0x0dce56a50b39…",2023-12-21 10:57:22,254908,190000000,0.000048,1,"""{""p"":""zrc-20"",…","""zrc-20""","""transfer""","""sync""","""2952""",null,null,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
24269907,"""0xbe1c2d2a167a…","""0x646174613a2c…","""0xde404aaa7437…","""0x5ba5cf528bae…",2024-01-17 11:06:12,417820,100000000,0.000042,1,"""{""p"":""zrc-20"",…","""zrc-20""","""transfer""","""sync""","""24""",null,null,0
24269941,"""0xee9659c534e9…","""0x646174613a2c…","""0x508703228fed…","""0x5ba5cf528bae…",2024-01-17 11:06:48,417835,100000000,0.000042,1,"""{""p"":""zrc-20"",…","""zrc-20""","""transfer""","""sync""","""124""",null,null,0
25022829,"""0x38a333db012c…","""0x646174613a2c…","""0x9be02d4c7987…","""0x3551bab7c6c8…",2024-01-26 14:18:53,275107,100000000,0.000028,1,"""{""p"":""zrc-20"",…","""zrc-20""","""transfer""","""sync""","""980""",null,null,0


In [86]:
for row in transfer_sync.iter_rows(named=True):
    tx_hash = row['tx_hash']
    tx_data = w3.eth.get_transaction_receipt(tx_hash)
    row['value'] = int(tx_data['logs'][0]['data'].hex(), 16) / 1e18

In [63]:
def calculate_value(tx_hash):
    tx_data = w3.eth.get_transaction_receipt(tx_hash)
    return int(tx_data['logs'][0]['data'].hex(), 16) / 1e18

# Apply the function to create a new column 'value' in the DataFrame
transfer_sync = transfer_sync.with_columns(
    pl.col('tx_hash').apply(calculate_value).alias('value')
)

/var/folders/y_/27jchs_x56gfpv07wkchjxj40000gn/T/ipykernel_1638/3978057001.py:7: DeprecationWarning:

`apply` is deprecated. It has been renamed to `map_elements`.



In [71]:
def update_amt(amt_str):
    if (amt_str == '1e6'):
        return '10000000'
    return (amt_str)

# Apply the function to create a new column 'value' in the DataFrame
transfer_sync = transfer_sync.with_columns(
    pl.col('amt').apply(update_amt).alias('amt')
)

transfer_sync = transfer_sync.with_columns(pl.col('amt').cast(pl.Int32))


/var/folders/y_/27jchs_x56gfpv07wkchjxj40000gn/T/ipykernel_1638/64109395.py:8: DeprecationWarning:

`apply` is deprecated. It has been renamed to `map_elements`.



In [73]:
transfer_sync = transfer_sync.with_columns((pl.col('value')*4000 / pl.col('amt')).alias('value_USD'))


In [86]:
transfer_sync

block_number,tx_hash,tx_input_data,issuer,receiver,timestamp,gas_used,gas_effective_price,fees,tx_status,decoded_input_data,p,op,tick,amt,tx,price,value,value_USD
i64,str,str,str,str,datetime[μs],i64,i64,f64,i64,str,str,str,str,i32,str,str,f64,f64
22048559,"""0xcad1a24e5e97…","""0x646174613a2c…","""0x4749ab4b7b47…","""0xb14f884adb66…",2023-12-21 10:40:14,248010,190000000,0.000047,1,"""{""p"":""zrc-20"",…","""zrc-20""","""transfer""","""sync""",480,null,null,0.000099,0.000824
22048747,"""0x9ce1f100c72a…","""0x646174613a2c…","""0x0a167d46ddf4…","""0xd7fdc1c61b53…",2023-12-21 10:43:28,249245,190000000,0.000047,1,"""{""p"":""zrc-20"",…","""zrc-20""","""transfer""","""sync""",4,null,null,0.000099,0.099103
22049338,"""0x5ddac4627c80…","""0x646174613a2c…","""0x484da9bda5e0…","""0x484da9bda5e0…",2023-12-21 10:53:43,253708,190000000,0.000048,1,"""{""p"":""zrc-20"",…","""zrc-20""","""transfer""","""sync""",1,null,null,0.000073,0.293006
22049457,"""0x103b5dffcc19…","""0x646174613a2c…","""0x35839602105d…","""0x484da9bda5e0…",2023-12-21 10:55:48,254897,190000000,0.000048,1,"""{""p"":""zrc-20"",…","""zrc-20""","""transfer""","""sync""",1,null,null,0.000074,0.294054
22049549,"""0xd4218e38f169…","""0x646174613a2c…","""0x3523f08e37d7…","""0x0dce56a50b39…",2023-12-21 10:57:22,254908,190000000,0.000048,1,"""{""p"":""zrc-20"",…","""zrc-20""","""transfer""","""sync""",2952,null,null,0.000102,0.000139
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
24269907,"""0xbe1c2d2a167a…","""0x646174613a2c…","""0xde404aaa7437…","""0x5ba5cf528bae…",2024-01-17 11:06:12,417820,100000000,0.000042,1,"""{""p"":""zrc-20"",…","""zrc-20""","""transfer""","""sync""",24,null,null,0.000062,0.010382
24269941,"""0xee9659c534e9…","""0x646174613a2c…","""0x508703228fed…","""0x5ba5cf528bae…",2024-01-17 11:06:48,417835,100000000,0.000042,1,"""{""p"":""zrc-20"",…","""zrc-20""","""transfer""","""sync""",124,null,null,0.000087,0.002809
25022829,"""0x38a333db012c…","""0x646174613a2c…","""0x9be02d4c7987…","""0x3551bab7c6c8…",2024-01-26 14:18:53,275107,100000000,0.000028,1,"""{""p"":""zrc-20"",…","""zrc-20""","""transfer""","""sync""",980,null,null,0.000067,0.000275


In [89]:
transfer_sync = transfer_sync.with_columns((pl.col('timestamp').cast(pl.Date)).alias('date'))
transfer_sync

block_number,tx_hash,tx_input_data,issuer,receiver,timestamp,gas_used,gas_effective_price,fees,tx_status,decoded_input_data,p,op,tick,amt,tx,price,value,value_USD,date
i64,str,str,str,str,datetime[μs],i64,i64,f64,i64,str,str,str,str,i32,str,str,f64,f64,date
22048559,"""0xcad1a24e5e97…","""0x646174613a2c…","""0x4749ab4b7b47…","""0xb14f884adb66…",2023-12-21 10:40:14,248010,190000000,0.000047,1,"""{""p"":""zrc-20"",…","""zrc-20""","""transfer""","""sync""",480,null,null,0.000099,0.000824,2023-12-21
22048747,"""0x9ce1f100c72a…","""0x646174613a2c…","""0x0a167d46ddf4…","""0xd7fdc1c61b53…",2023-12-21 10:43:28,249245,190000000,0.000047,1,"""{""p"":""zrc-20"",…","""zrc-20""","""transfer""","""sync""",4,null,null,0.000099,0.099103,2023-12-21
22049338,"""0x5ddac4627c80…","""0x646174613a2c…","""0x484da9bda5e0…","""0x484da9bda5e0…",2023-12-21 10:53:43,253708,190000000,0.000048,1,"""{""p"":""zrc-20"",…","""zrc-20""","""transfer""","""sync""",1,null,null,0.000073,0.293006,2023-12-21
22049457,"""0x103b5dffcc19…","""0x646174613a2c…","""0x35839602105d…","""0x484da9bda5e0…",2023-12-21 10:55:48,254897,190000000,0.000048,1,"""{""p"":""zrc-20"",…","""zrc-20""","""transfer""","""sync""",1,null,null,0.000074,0.294054,2023-12-21
22049549,"""0xd4218e38f169…","""0x646174613a2c…","""0x3523f08e37d7…","""0x0dce56a50b39…",2023-12-21 10:57:22,254908,190000000,0.000048,1,"""{""p"":""zrc-20"",…","""zrc-20""","""transfer""","""sync""",2952,null,null,0.000102,0.000139,2023-12-21
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
24269907,"""0xbe1c2d2a167a…","""0x646174613a2c…","""0xde404aaa7437…","""0x5ba5cf528bae…",2024-01-17 11:06:12,417820,100000000,0.000042,1,"""{""p"":""zrc-20"",…","""zrc-20""","""transfer""","""sync""",24,null,null,0.000062,0.010382,2024-01-17
24269941,"""0xee9659c534e9…","""0x646174613a2c…","""0x508703228fed…","""0x5ba5cf528bae…",2024-01-17 11:06:48,417835,100000000,0.000042,1,"""{""p"":""zrc-20"",…","""zrc-20""","""transfer""","""sync""",124,null,null,0.000087,0.002809,2024-01-17
25022829,"""0x38a333db012c…","""0x646174613a2c…","""0x9be02d4c7987…","""0x3551bab7c6c8…",2024-01-26 14:18:53,275107,100000000,0.000028,1,"""{""p"":""zrc-20"",…","""zrc-20""","""transfer""","""sync""",980,null,null,0.000067,0.000275,2024-01-26


In [119]:
fig = go.Figure(layout=layout)


temp_df = transfer_sync.filter((pl.col('value_USD')<1))

fig.add_trace(go.Scatter(x=temp_df['timestamp'], y=temp_df['value_USD'], mode='markers', name='Price'))

# Update layout
fig.update_layout(xaxis_title='Transaction Timestamp',
                  yaxis_title='Price (USD)')

fig.update_layout(yaxis=dict(range=[-0.01, 0.8]))

fig.write_image(plots_dir+"transfersync_blue.pdf",
                width=1540, height=380, scale=1)

fig.show()

In [199]:
df = transfer_sync.select(pl.std("value_USD"))
df["value_USD"]

df = transfer_sync.select(pl.median("value_USD"))
df["value_USD"]

df = transfer_sync.select(pl.mean("value_USD"))
df["value_USD"]

df = transfer_sync.select(pl.min("value_USD"))
df["value_USD"]

df = transfer_sync.select(pl.max("value_USD"))
df["value_USD"]

value_USD
f64
0.406307


In [ ]:
data = transfer_sync.group_by('date').agg(pl.col("value_USD").sum())
data = data.sort(pl.col("date"))
data


In [156]:
fig = go.Figure(layout=layout)


#data = transfer_sync.group_by('date').agg(pl.col("value_USD").sum())
#data = data.sort(pl.col("date"))
#fig.add_trace(go.Scatter(x=data['date'], y=data['value_USD'], line=dict(
#    color=colors['light_purple'], width=4), mode='lines', name='Total Volume'))

data = transfer_sync.group_by('date').agg(pl.col("value_USD").median())
data = data.sort(pl.col("date"))
fig.add_trace(go.Scatter(x=data['date'], y=data['value_USD'], line=dict(
    color=colors['brown'], width=2), mode='lines', name='Median Price'))

data = transfer_sync
data = data.sort(pl.col("date"))
fig.add_trace(go.Box(x=data['date'], y=data['value_USD'],
              name='Transaction Price', marker_color=colors['green'], line=dict(width=1.5), whiskerwidth=0.5, fillcolor=colors['white']))

# Update layout
fig.update_layout(xaxis_title='Transaction Date',
                  yaxis_title='Price (USD)', legend=dict(xanchor='center',
                                                  x=0.5, y=1.02, orientation='h'))

fig.update_layout(yaxis=dict(range=[-0.01, 0.9]))
#fig.update_yaxes(type='log', range=[-11, 2])


fig.write_image(plots_dir+"transfersync.pdf",
                width=770, height=380, scale=1) #1540
fig.show()

In [ ]:
from plot_utils import get_plotly_layout
from plot_utils import colors
width, height = 800, 450
fig = go.Figure(layout=get_plotly_layout(width=width, height=height))


data = votes_cost_df.groupby('proposalId').tx_fee_usd.sum()
fig.add_trace(go.Scatter(x=data.index, y=data, line=dict(
    color=colors['light_purple'], width=4), mode='lines', name='total cost'))

data = votes_cost_df.groupby('proposalId').tx_fee_usd.median()
fig.add_trace(go.Scatter(x=data.index, y=data, line=dict(
    color=colors['brown'], width=2), mode='lines', name='median cost'))


data = votes_cost_df.query(
    'proposalId not in @proposals_defeated_df.proposalId')
fig.add_trace(go.Box(x=data.proposalId, y=data.tx_fee_usd,
              name='Executed', marker_color=colors['green'], line=dict(width=1.5), whiskerwidth=0.5, fillcolor=colors['white']))

data = votes_cost_df.query(
    'proposalId in @proposals_defeated_df.proposalId')
fig.add_trace(go.Box(x=data.proposalId, y=data.tx_fee_usd,
              name='Defeated', marker_color=colors['red'], line=dict(width=1.5), whiskerwidth=0.5, fillcolor=colors['white']))

data = votes_cost_df.query(
    'proposalId in @proposals_canceled_df.proposalId')
fig.add_trace(go.Box(x=data.proposalId, y=data.tx_fee_usd,
              name='Cancelled', marker_color=colors['blue'], line=dict(width=1.5), whiskerwidth=0.5, fillcolor=colors['white']))

fig.update_layout(yaxis_title="Voting cost (in USD)", xaxis_title="Proposal ID", template='simple_white',
                  font=dict(size=18, family='Clear Sans'), xaxis=dict(tickmode='linear', tick0=0, dtick=10),
                  legend=dict(xanchor='center', x=0.5, y=1.15, orientation='h'))


fig.update_xaxes(minor_ticks="inside", showgrid=True,
                 griddash='dash', minor_griddash="dot")
fig.update_yaxes(minor_ticks="inside", showgrid=True,
                 griddash='dash', minor_griddash="dot")
fig.update_yaxes(type='log')

fig.write_image(plots_dir+"voting_cost_per_proposal.pdf",
                width=1540, height=380, scale=1)
fig.show()